In [1]:
import numpy as np
import pandas as pd
import pickle
from helper import *

In [2]:
##  Creating the alpha matrix: expenditure share on good i in region r:
##  (VPA + VGA+ S_i VFA)/GDP

In [3]:
username = 'ozaltun'

In [4]:
root = '/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/output/baseview/{}.csv'

vpa =  pd.read_csv(root.format('SF02'), index_col=0, header=1, low_memory=False)
vga =  pd.read_csv(root.format('SF03'), index_col=0, header=1, low_memory=False)
vfa =  pd.read_csv(root.format('SF01'), index_col=0, header=1, low_memory=False)

GDP =  pd.read_csv(root.format('MSUM'), index_col=0, header=1, low_memory=False)

comm_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/comm_2_num.pickle')
reg_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/reg_2_num.pickle')
prodcomm_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/prodcomm_2_num.pickle')



/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
## SUM OVER DIR AND PURCHVALUE BITCH!

In [6]:
vpa.drop(['DIR', 'PURCHVALUE'], axis=1, inplace=True)
vga.drop(['DIR', 'PURCHVALUE'], axis=1, inplace=True)

# Lets just get the TRAD_COMM values
demd_2_comm = {i:i for i in list(comm_2_num.keys())}
vfa['TRAD_COMM'] = vfa.DEMD_COMM.map(demd_2_comm)
vfa = vfa[vfa.TRAD_COMM.isna() == False]
vfa.drop(['DIR', 'PURCHVALUE','DEMD_COMM'], axis=1, inplace=True)


In [7]:
# vpa = vpa.groupby(['TRAD_COMM','REG']).sum().reset_index()
# vga = vga.groupby(['TRAD_COMM','REG']).sum().reset_index()
# vfa = vfa.groupby( ['TRAD_COMM','PROD_COMM','REG']).sum().reset_index()

In [8]:
vpa_np = convertRow2Matrix(vpa, ['TRAD_COMM','REG'], [comm_2_num, reg_2_num])
vga_np = convertRow2Matrix(vga, ['TRAD_COMM','REG'], [comm_2_num, reg_2_num])
vfa_np = convertRow2Matrix(vfa, ['TRAD_COMM','PROD_COMM','REG'], [comm_2_num, prodcomm_2_num, reg_2_num])

Number of empty values:  []
Number of empty values:  []
Number of empty values:  []


In [9]:
## Now get GDP
GDP = GDP[GDP.SUMREP == 'GDP']

In [10]:
GDP.drop(['SUMREP'], axis=1, inplace=True)

In [11]:
# gdp_np = convertRow2Matrix(GDP, ['REG'], [reg_2_num])
GDP['REG_Number'] = GDP.REG.map(reg_2_num)

In [12]:
GDP.set_index('REG_Number', inplace=True)
GDP.sort_index(inplace=True)

In [13]:
gdp_np = GDP.Value.to_numpy()

In [14]:
print(vpa_np.shape, vga_np.shape, vfa_np.shape,gdp_np.shape )

(57, 134) (57, 134) (57, 58, 134) (134,)


In [ ]:
## Similar to the TODO with e's, this is summing to more than one for
## each row. We will bootsrap it now, by just dividing it by "synthetic
## GDP

In [15]:
syn_gdp = (vpa_np.sum(axis=0) + vga_np.sum(axis=0) + vfa_np.sum(axis=1).reshape((57, 134)).sum(axis=0)).reshape((1, 134))

In [16]:
alpha = (vpa_np + vga_np + vfa_np.sum(axis=1).reshape((57, 134)))/syn_gdp

In [19]:
alpha.sum(axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [20]:
root_matrix = '/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/matrices/alpha.pickle'
save_obj(alpha, root_matrix)

In [21]:
alpha[alpha>1]

array([], dtype=float64)